In [16]:
import logging
from functools import partial

from collections import OrderedDict
from typing import Sequence, Any, Iterable, Optional, List
import numpy as np
# import click
# import click_log
from tqdm import tqdm_notebook as tqdm
import torch
import torch.nn as nn
import torch.nn.functional as tnnf
from torchvision.datasets.mnist import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import StepLR
from tt_model import *


In [24]:
config = {
    'resize_shape': (32, 32),
    
    'in_factors': (4, 4, 4, 4, 4),
    'l1_ranks': (8, 8, 8, 8),
    'hidd_out_factors': (2, 2, 2, 2, 2),
    'ein_string1': "nabcde,aoiv,bijw,cjkx,dkly,elpz",
    
    'hidd_in_factors': (4, 8),
    'l2_ranks': (16,),
    'out_factors': (5, 2),
    'ein_string2': 'nab,aoix,bipy',
}

parameters_config = {
    'batch_size': 10000,
    'train_dataset_size': 40000,
    'device': torch.device('cuda'),
    'learning_rate': 1e-3,
    'n_epochs': 100
}

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
        
model_cfg = AttrDict(config)
params_cfg = AttrDict(parameters_config)


In [18]:
model = TTModel(model_cfg)
model = model.to(params_cfg.device)


In [25]:

MNIST_DATASET_SIZE = 60000
NUM_LABELS = 10

MNIST_TRANSFORM = transforms.Compose((
    transforms.Pad(2),
    transforms.ToTensor(),
    transforms.Normalize((0.1,), (0.2752,))
))




dataset = MNIST('mnist', train=True, download=True, transform=MNIST_TRANSFORM)
assert len(dataset) == MNIST_DATASET_SIZE
train_dataset, val_dataset = random_split(
    dataset, (params_cfg.train_dataset_size, MNIST_DATASET_SIZE - params_cfg.train_dataset_size)
)

train_loader, val_loader = (
    DataLoader(
        dataset_, batch_size=params_cfg.batch_size, shuffle=True, pin_memory=(params_cfg.device.type == "cuda")
    )
    for dataset_ in (train_dataset, val_dataset)
)




In [26]:
def acc(model, loader):
    accs = []
    with torch.no_grad():
        for b, gt in tqdm(loader):
            out = model(b.to(params_cfg.device)).argmax(1).cpu().numpy()
            gt = gt.numpy()
            accs.append(sum(out == gt) / len(out))
    return sum(accs) / len(accs) 



lf = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=params_cfg.learning_rate)
for ep in range(params_cfg.n_epochs):
    for b, gt in tqdm(train_loader):
        
        optimizer.zero_grad()
        out = model(b.to(params_cfg.device))
        loss = lf(out, gt.to(params_cfg.device))
        loss.backward()
        optimizer.step()
    
    print(acc(model, val_loader))
    

0.88965


0.8976


0.9058999999999999


0.9063


0.9060999999999999


...

0.935


0.9351


0.93565


0.9347


0.9359


In [ ]:
acc(model, val_loader)